In [ ]:
from selenium import webdriver
import time
from urllib import parse
from bs4 import BeautifulSoup as bs

In [ ]:
# ① 크롬 브라우저 열기
driver = webdriver.Chrome('chromedriver.exe')

# 인스타그램 접속하기
driver.get('http://www.instargram.com')
time.sleep(3)

######## 인스타 계정 로그인이 필요합니다 #########
# 예제 4-2 selenium으로 URL 접속하기 - 2

email = '계정 아이디' 
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '계정 비밀번호' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)


In [50]:
def instagramfunc(word, target):
    def insta_searching(word):
        url = "http://www.instagram.com/explore/tags/" + word
        return url
    
    url = insta_searching(word)
    driver.get(url)
    time.sleep(3)

    # 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기
    def select_first(driver):
        first = driver.find_element_by_css_selector("div._9AhH0")
        first.click()
        time.sleep(3)
    select_first(driver)

    results = []
    
     
    import re
    from bs4 import BeautifulSoup

    def get_content(driver):
        # ① 현재 페이지 html 정보 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # ② 본문 내용 가져오기
        try:
            content = soup.select('div.C4VMK > span')[0].text
        except:
            content = ''

        # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
        tags = re.findall(r'#[^\s#,\\]+', content)  

        # 댓글 크롤링....
        print('댓글')
        try:
            comment = soup.select('div.C4VMK > span')[1].text
        except:
            comment = ''
        print(comment)

        ctags = re.findall(r'#[^\s#,\\]+', comment)


        # 본문 + 댓글 태그 합치기 / 중복 제거
        TAG = tags + ctags
        TAG = set(TAG)
        TAG = list(TAG)
        print(TAG)

        # ⑦ 수집한 정보 저장하기
        data = [TAG]
        return data

    # get_content(driver)

    def move_next(driver,sleeptime):
        try:
            right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
            right.click()
            time.sleep(sleeptime)
        except:
            return

    # move_next(driver)
   
    import random
    for i in range(target):
        if i < 500:
            sleeptime = random.randrange(6, 14) / 3
        elif i < 1000:
            sleeptime = random.randrange(8, 23) / 4
        elif i < 1500:
            sleeptime = random.randrange(6, 20) / 3
        elif i < 2000:
            sleeptime = random.randrange(8, 23) / 4
        elif i < 2500:
            sleeptime = random.randrange(10, 32) / 5
        elif i < 3000:
            sleeptime = random.randrange(6, 14) / 3
        
        print(i, ' ', sleeptime)
        # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
        try:
            data = get_content(driver)    # 게시글 정보 가져오기
            results.append(data)
            move_next(driver,sleeptime)
        except:
            time.sleep(sleeptime)
            move_next(driver)


    print(results[:2])

    import pandas as pd

    results_df = pd.DataFrame(results)
    results_df.columns = ['tag']
    results_df.to_csv('travel_ex.csv', encoding='utf-8')

    # 예제 4-8 여러 개의 저장파일 통합하기
    travel_insta_df = pd.DataFrame( [ ] )

    #folder = './files/'
    f_list = ['travel_ex.csv']
    for fname in f_list:
        fpath =  fname
        temp = pd.read_csv(fpath)
        travel_insta_df = travel_insta_df.append(temp)

    travel_insta_df.columns =['scrollnum', 'tag']

    # 예제 4-9 중복 데이터 제거하고 저장하기
    #travel_insta_df.drop_duplicates(subset = ['content'] , inplace = True)
    travel_insta_df.to_csv('insta_{}.csv', index = False, encoding = 'utf8')